In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from matplotlib.pylab import plt
from pandas import DataFrame
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def plot_scatter(xy, labels, title=""):
    # scatter plot, dots colored by class value
    df = DataFrame(dict(x=xy[:,0], y=xy[:,1], label=labels))
    colors = {1:'red', 0:'blue'}
    fig, ax = plt.subplots(figsize=(5,5))
    grouped = df.groupby('label')
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, \
                   color=colors[key], edgecolor='k', alpha=0.5)
    plt.axis('equal')
    plt.title(title)
    plt.grid()
    plt.show()

In [ ]:
# generate 2d classification dataset
xy, labels = make_blobs(n_samples=400, center_box=(-1,1), centers=6,cluster_std=0.2, random_state=20)
labels = labels % 2
plot_scatter(xy, labels)

In [ ]:
features = np.array([[xval, yval] for xval, yval in xy])
labels = labels.reshape(-1,1)

In [ ]:
features_train, features_test, labels_train, labels_test  = train_test_split(features, labels, test_size = 0.3)

In [ ]:
plot_scatter(features_train, labels_train.flatten())

In [ ]:
plot_scatter(features_test, labels_test.flatten())

In [ ]:
def plot_model(sess, model, titles):
    train_and_test = [(features_train, labels_train), (features_test, labels_test)]
    xx, yy = np.meshgrid(np.linspace(-1.5,1.5), np.linspace(-1.5,1.5))
    prediction = sess.run(model, feed_dict={x: np.array([[xxval, yyval] for xxval, yyval in zip(xx.flatten(), yy.flatten())])})
    Z = prediction.reshape(xx.shape)
    colors = {1:'red', 0:'blue'}
    _, axes = plt.subplots(1, 2, figsize=(8, 4))
    for (xy_, labels_), ax, title in zip(train_and_test, axes, titles):
        df = DataFrame(dict(x=xy_[:,0], y=xy_[:,1], label=labels_.flatten()))
        ax.contourf(xx, yy, Z, cmap='coolwarm', alpha=.9,)
        grouped = df.groupby('label')
        for key, group in grouped:
            group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key], edgecolor='k')
        ax.set_xlim([-1.3, 1.3])
        ax.set_ylim([-1.3, 1.3])
        ax.grid(linestyle='--')
        ax.set_title(title)
    plt.show()

## Problem 1

In [ ]:
# TODO 1 Hyper-Parameters
num_hidden1 = 10
num_hidden2 = 4
learning_rate = 1E-2
MaxEpoch = 1500

# Model
tf.set_random_seed(180417)
x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

hidden1 = tf.layers.dense(x, units=num_hidden1, use_bias=True, activation=tf.nn.sigmoid)
hidden2 = tf.layers.dense(hidden1, units=num_hidden2, use_bias=True, activation=tf.nn.sigmoid)
yhat = tf.layers.dense(hidden2, units=1, use_bias=True, activation=tf.nn.sigmoid)

loss = tf.reduce_mean(-y * tf.log(yhat) - (1 - y) * tf.log(1 - yhat))

train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
losses1 = []
for epoch in range(MaxEpoch):
    sess.run(train, feed_dict={x:features_train, y:labels_train})
    train_loss = sess.run(loss, feed_dict={x:features_train, y:labels_train})
    test_loss = sess.run(loss, feed_dict={x:features_test, y:labels_test})
    losses1.append([train_loss, test_loss])
    if epoch % 200 == 0:
        print(epoch, train_loss, test_loss)
        
losses1 = np.array(losses1)

In [ ]:
plt.plot(losses1[:, 0], label='train')
plt.plot(losses1[:, 1], label='test')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plot_model(sess, yhat, ["Train", "Test"])

## Problem 2

In [ ]:
# TODO 2 Hyper-Parameters
num_hidden1 = None
num_hidden2 = None
learning_rate = None
MaxEpoch = None

# Model
tf.set_random_seed(180417)
x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

hidden1 = tf.layers.dense(x, units=num_hidden1, use_bias=True, activation=tf.nn.sigmoid)
hidden2 = tf.layers.dense(hidden1, units=num_hidden2, use_bias=True, activation=tf.nn.sigmoid)
yhat = tf.layers.dense(hidden2, units=1, use_bias=True, activation=tf.nn.sigmoid)

loss = tf.reduce_mean(-y * tf.log(yhat) - (1 - y) * tf.log(1 - yhat))

train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

losses2 = []
for epoch in range(MaxEpoch):
    sess.run(train, feed_dict={x:features_train, y:labels_train})
    train_loss = sess.run(loss, feed_dict={x:features_train, y:labels_train})
    test_loss = sess.run(loss, feed_dict={x:features_test, y:labels_test})
    losses2.append([train_loss, test_loss])
    if epoch % 200 == 0:    
        print(epoch, train_loss, test_loss)
losses2 = np.array(losses2)

In [ ]:
plt.plot(losses2[:, 0], label='train')
plt.plot(losses2[:, 1], label='test')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plot_model(sess, yhat, ["Train", "Test"])